In [4]:
import numpy as np
from timeit import default_timer as timer
from matplotlib import pyplot
import math
from numbapro import vectorize #from accelerate import vectorize

In [5]:
@vectorize(['float64(float64, float64, float64, float64, float64)'],target='gpu')
def step(price, dt, c0, c1, noise):
    return price * math.exp(c0 * dt + c1 * noise)

def montecarlo(paths, dt, interest, volatility):
    c0 = interest - 0.5 * volatility ** 2
    c1 = volatility * np.sqrt(dt)
    
    for j in xrange(1, paths.shape[1]): #for each time step
        prices = paths[:, j - 1]
        # gaussian noises for simulation
        noises = np.random.normal(0., 1., prices.size)
        #simulate
        paths[:, j] = step(prices, dt, c0, c1, noises)
        
#Stock information parameters
StockPrice = 20.83
StrikePrice = 21.50
Volatility = 0.021
InterestRate = 0.20
Maturity = 5.  / 12.

NumPath = 3000000
NumStep = 100

MAX_PATH_IN_PLOT = 50

def driver(pricer, do_plot=False):
    paths = np.zeros((NumPath, NumStep + 1), order='F')
    paths[:, 0] = StockPrice
    DT = Maturity / NumStep
    
    ts = timer()
    pricer(paths, DT, InterestRate, Volatility)
    te = timer()
    elapsed = te - ts
    
    ST = paths[:, -1]
    PaidOff = np.maximum(paths[:, -1] - StrikePrice, 0)
    print 'Result'
    fmt = '%20s: %s'
    print fmt % ('stock price', np.mean(ST))
    print fmt % ('standard error', np.std(ST) / np.sqrt(NumPath))
    print fmt % ('paid off', np.mean(PaidOff))
    optionprice = np.mean(PaidOff) * np.exp(-InterestRate * Maturity)
    print fmt % ('option price', optionprice)
    
    print 'Performance'
    NumCompute = NumPath * NumStep
    print fmt % ('Mstep/second', '%.2f' % (NumCompute / elapsed / 1e6))
    print fmt % ('time elapsed', '%.3fs' % (te - ts))
    
    if do_plot:
        pathct = min(NumPath, MAX_PATH_IN_PLOT)
        for i in xrange(pathct):
                 pyplot.plot(paths[i])
        print 'Plotting %d/%d paths' % (pathct, NumPath)
        pyplot.show()
                 
import cProfile as profile
profile.run("driver(montecarlo, do_plot=True)", sort="time")

Result
         stock price: 22.6400763691
      standard error: 0.000177264661069
            paid off: 1.14008160704
        option price: 1.04892571478
Performance
        Mstep/second: 21.73
        time elapsed: 13.809s
Plotting 50/3000000 paths
         348178 function calls (344838 primitive calls) in 17.144 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100   11.868    0.119   11.868    0.119 {method 'normal' of 'mtrand.RandomState' objects}
        1    2.808    2.808    3.044    3.044 {built-in method exec_}
     3195    1.342    0.000    1.344    0.000 driver.py:215(safe_cuda_api_call)
        1    0.419    0.419   13.808   13.808 <ipython-input-5-c54f549fffa8>:5(montecarlo)
        1    0.080    0.080    0.081    0.081 backend_qt5.py:123(_create_qApp)
     3367    0.035    0.000    0.035    0.000 weakref.py:282(__init__)
     2436    0.028    0.000    0.028    0.000 {method 'reduce' of 'numpy.ufunc' objec

In [2]:
numbapro.check_cuda()

------------------------------libraries detection-------------------------------
Finding cublas
	located at /home/john/anaconda2/lib/libcublas.so.7.0.28
	trying to open library...	ok
Finding cusparse
	located at /home/john/anaconda2/lib/libcusparse.so.7.0.28
	trying to open library...	ok
Finding cufft
	located at /home/john/anaconda2/lib/libcufft.so.7.0.35
	trying to open library...	ok
Finding curand
	located at /home/john/anaconda2/lib/libcurand.so.7.0.28
	trying to open library...	ok
Finding nvvm
	located at /home/john/anaconda2/lib/libnvvm.so.3.0.0
	trying to open library...	ok
	finding libdevice for compute_20...	ok
	finding libdevice for compute_30...	ok
	finding libdevice for compute_35...	ok
-------------------------------hardware detection-------------------------------
Found 1 CUDA devices
id 0        GeForce GTX 870M                              [SUPPORTED]
                      compute capability: 3.0
                           pci device id: 0
                              

True